# Supervised Learning Capstone Project - Tree Methods Focus 



## Make sure to review the introduction video to understand the 3 ways of approaching this project exercise!

----

**Ways to approach the project:**
   1. Open a new notebook, read in the data, and then analyze and visualize whatever you want, then create a predictive model.
   2. Use this notebook as a general guide, completing the tasks in bold shown below.
   3. Skip to the solutions notebook and video, and treat project at a more relaxing code along walkthrough lecture series.

------
------

## GOAL: Create a model to predict whether or not a customer will Churn .

----
----


## Complete the Tasks in Bold Below!

## Part 0: Imports and Read in the Data

**TASK: Run the filled out cells below to import libraries and read in your data. The data file is "Telco-Customer-Churn.csv"**

In [1]:
# RUN THESE CELLS TO START THE PROJECT!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('BASES/Telco-Customer-Churn.csv')

In [ ]:
df.head()

## Part 1: Quick Data Check

**TASK: Confirm quickly with .info() methods the datatypes and non-null values in your dataframe.**

In [ ]:
# CODE HERE

df.info()

**TASK: Get a quick statistical summary of the numeric columns with .describe() , you should notice that many columns are categorical, meaning you will eventually need to convert them to dummy variables.**

In [ ]:
# CODE HERE

df.describe()

# Part 2:  Exploratory Data Analysis

## General Feature Exploration

**TASK: Confirm that there are no NaN cells by displaying NaN values per feature column.**

In [ ]:
# CODE HERE

df.isnull().sum()

**TASK:Display the balance of the class labels (Churn) with a Count Plot.**

In [ ]:
# CODE HERE

sns.countplot(data=df,
              x='Churn')

<img src='fig1.png' >

**TASK: Explore the distrbution of TotalCharges between Churn categories with a Box Plot or Violin Plot.**

In [ ]:
# CODE HERE

plt.figure(figsize=(15, 10))
sns.violinplot(data=df,
               y='TotalCharges',
               #hue='test preparation course',
               x='Churn',
               palette='Set2')


<img src='fig2.png' >

**TASK: Create a boxplot showing the distribution of TotalCharges per Contract type, also add in a hue coloring based on the Churn class.**

In [ ]:
#CODE HERE

plt.figure(figsize=(15, 10))
sns.boxplot(data=df,
            y='TotalCharges',
            x='Contract',
            hue='Churn',
            palette='Set2')
plt.legend(bbox_to_anchor=(1.05, 0.5))

<img src='fig3.png' >

**TASK: Create a bar plot showing the correlation of the following features to the class label. Keep in mind, for the categorical features, you will need to convert them into dummy variables first, as you can only calculate correlation for numeric features.**

    ['gender', 'SeniorCitizen', 'Partner', 'Dependents','PhoneService', 'MultipleLines', 
     'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'InternetService',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

***Note, we specifically listed only the features above, you should not check the correlation for every feature, as some features have too many unique instances for such an analysis, such as customerID***

In [ ]:


my_object_df = df.select_dtypes(include='object')
my_numeric_df = df.select_dtypes(exclude='object')


y = df['Churn']

my_object_df.drop(['customerID', 'Churn'], axis=1, inplace=True)

df_object_dummies = pd.get_dummies(my_object_df,
                                   drop_first=True)


In [ ]:
df = pd.concat([my_numeric_df, df_object_dummies, y],
                     axis=1)

In [ ]:
df_object_dummies.columns

In [ ]:
df['Churn_num'] = (

    np.where(df['Churn'] == 'No', 0, 1)
    ''
)

In [ ]:
corr = df.corr()

In [ ]:
corr = corr['Churn_num'].sort_values()
corr

In [ ]:
corr = corr[corr.index.values != 'Churn_num']

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(x=corr.index,
            y=corr.values)
plt.xticks(rotation=90)


<img src='figbar.png'>

In [ ]:
# CODE HERE

sns.countplot(data=df,
              hue='InternetService_Fiber optic',
              x='Churn')


---
---

# Part 3: Churn Analysis

**This section focuses on segementing customers based on their tenure, creating "cohorts", allowing us to examine differences between customer cohort segments.**

**TASK: What are the 3 contract types available?**

In [2]:
# CODE HERE

df = pd.read_csv('BASES/Telco-Customer-Churn.csv')

df['Contract'].unique()

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

**TASK: Create a histogram displaying the distribution of 'tenure' column, which is the amount of months a customer was or has been on a customer.**

In [ ]:
#CODE HERE

plt.figure(figsize=(10, 4))
sns.histplot(data=df, 
             bins=40,
             x='tenure')

<img src="fig5.png">

**TASK: Now use the seaborn documentation as a guide to create histograms separated by two additional features, Churn and Contract.**

In [ ]:
#CODE HERE

g = sns.displot(data=df, 
                x='tenure', 
                col='Contract', 
                row='Churn',
                common_bins=False)

<img src="fig6.png">

**TASK: Display a scatter plot of Total Charges versus Monthly Charges, and color hue by Churn.**

In [ ]:
#CODE HERE

plt.figure(figsize=(15, 10))
sns.scatterplot(data=df,
                hue='Churn',
                x='MonthlyCharges',
                y='TotalCharges')

<img src='fig7.png'>

### Creating Cohorts based on Tenure

**Let's begin by treating each unique tenure length, 1 month, 2 month, 3 month...N months as its own cohort.**

**TASK: Treating each unique tenure group as a cohort, calculate the Churn rate (percentage that had Yes Churn) per cohort. For example, the cohort that has had a tenure of 1 month should have a Churn rate of 61.99%. You should have cohorts 1-72 months with a general trend of the longer the tenure of the cohort, the less of a churn rate. This makes sense as you are less likely to stop service the longer you've had it.**

In [ ]:
#CODE HERE

df_agg = (

    df.groupby(['tenure', 'Churn'], as_index = False).agg({'customerID':'count'})
)


df_pt = pd.pivot_table(df_agg, 
                      aggfunc = np.sum,
                      columns = ['Churn'],
                      index = ['tenure'],
                      values=['customerID'])

df_pt.columns = ['N', 'Y']

df_pt['CR'] = df_pt['Y'] / (df_pt['Y']  + df_pt['N'] )

df_pt

**TASK: Now that you have Churn Rate per tenure group 1-72 months, create a plot showing churn rate per months of tenure.**

In [ ]:
#CODE HERE


plt.figure(figsize=(15, 10))
plt.plot(df_pt.index, df_pt['CR'])
plt.ylabel('Churn Perfomance')
plt.xlabel('tenure')


<img src='fig9.png'>

### Broader Cohort Groups
**TASK: Based on the tenure column values, create a new column called Tenure Cohort that creates 4 separate categories:**
   * '0-12 Months'
   * '12-24 Months'
   * '24-48 Months'
   * 'Over 48 Months'    

In [3]:
# CODE HERE

df['Tenure Cohort'] = (

    np.where(df['tenure'] <= 12,
            '0-12 Months',
            np.where(df['tenure'] <= 24,
                     '12-24 Months',
                     np.where(df['tenure'] <= 48,
                              '24-48 Months',
                              'Over 48 Months')))
)

df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Tenure Cohort
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0-12 Months
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,24-48 Months
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,0-12 Months
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,24-48 Months
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,0-12 Months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No,12-24 Months
7028,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No,Over 48 Months
7029,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No,0-12 Months
7030,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes,0-12 Months


In [ ]:
df.columns

**TASK: Create a scatterplot of Total Charges versus Monthly Charts,colored by Tenure Cohort defined in the previous task.**

In [ ]:
#CODE HERE

plt.figure(figsize=(12, 8))
sns.scatterplot(data=df,
                hue='Tenure Cohort',
                x='MonthlyCharges',
                y='TotalCharges')

<img src='fig10.png'>

**TASK: Create a count plot showing the churn count per cohort.**

In [ ]:
# CODE HERE

plt.figure(figsize=(15, 8))
sns.countplot(data=df,
              x='Tenure Cohort',
              hue='Churn')

<img src='cplot.png'>

**TASK: Create a grid of Count Plots showing counts per Tenure Cohort, separated out by contract type and colored by the Churn hue.**

In [ ]:
#CODE HERE

g = sns.displot(data=df, 
                x='gender', 
                col='Contract',
                hue='Churn')

<img src='fig11.png'>

-----

# Part 4: Predictive Modeling

**Let's explore 4 different tree based methods: A Single Decision Tree, Random Forest, AdaBoost, Gradient Boosting. Feel free to add any other supervised learning models to your comparisons!**


## Single Decision Tree

**TASK : Separate out the data into X features and Y label. Create dummy variables where necessary and note which features are not useful and should be dropped.**

In [ ]:
#CODE HERE


my_object_df = df.select_dtypes(include='object')
my_numeric_df = df.select_dtypes(exclude='object')


y = df['Churn']

my_object_df.drop(['customerID', 'Churn'], axis=1, inplace=True)

df_object_dummies = pd.get_dummies(my_object_df,
                                   drop_first=True)

X = pd.concat([my_numeric_df, df_object_dummies], axis=1)


**TASK: Perform a train test split, holding out 10% of the data for testing. We'll use a random_state of 101 in the solutions notebook/video.**

In [4]:
#CODE HERE

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=101)

**TASK: Decision Tree Perfomance. Complete the following tasks:**
   1. Train a single decision tree model (feel free to grid search for optimal hyperparameters).
   2. Evaluate performance metrics from decision tree, including classification report and plotting a confusion matrix.
   2. Calculate feature importances from the decision tree.
   4. OPTIONAL: Plot your tree, note, the tree could be huge depending on your pruning, so it may crash your notebook if you display it with plot_tree.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
dt = DecisionTreeClassifier()


In [ ]:

param_grid = {'criterion':['gini', 'entropy', 'log_loss'], #gini
              'splitter':['best', 'random'], #best
              'min_samples_split':[1, 2, 5, 10, 15, 20, 40],
              'min_samples_leaf':[1, 2, 5, 10],
              'max_depth':[4, 5, 6, 8, 10, 15, 20]
             }


In [ ]:
X_train

In [ ]:
grid = GridSearchCV(dt, param_grid, cv=10)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
preds = grid.predict(X_test)

In [ ]:
grid.best_estimator_

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, accuracy_score

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
plot_confusion_matrix(grid, X_test, y_test)

In [ ]:
feats = pd.DataFrame(data=grid.best_estimator_.feature_importances_,
                     index=X.columns,
                     columns=['IMPO'])

In [ ]:
feats = feats.sort_values(by='IMPO')

In [ ]:
feats

In [ ]:
plt.figure(figsize=(15, 8))
sns.barplot(data=feats,
            y=feats.index,
            x=feats.IMPO)

In [ ]:
model = DecisionTreeClassifier(min_samples_split=40, splitter='random')

In [ ]:
model.fit(X_train, y_train)

In [ ]:
plot_tree(model, 
          feature_names=X.columns,
          filled=False)

<img src='hugetree.png'>

## Random Forest

**TASK: Create a Random Forest model and create a classification report and confusion matrix from its predicted results on the test set.**

In [ ]:
#CODE HERE


In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:

n_estimators = [64, 100, 128, 200]
max_features = [2, 3, 4]

bootstrap = [True, False]
oob_score = [True, False]

pg_rf = {'n_estimators':n_estimators,
         'max_features':max_features,
         'bootstrap':bootstrap,
         'max_depth':[3, 5, 8, 15],
         'criterion': ['gini', 'entropy'],
         #'min_samples_split':[1, 2, 4,],
         'oob_score':oob_score}

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
pg_rf

In [ ]:
grid = GridSearchCV(rfc, pg_rf, cv=10)

grid.fit(X_train, y_train)


In [ ]:

preds = grid.predict(X_test)

print(grid.best_estimator_)
print(classification_report(y_test, preds))

In [ ]:
plot_confusion_matrix(grid, X_test, y_test)

In [ ]:
#grid.oob_score_

## Boosted Trees

**TASK: Use AdaBoost or Gradient Boosting to create a model and report back the classification report and plot a confusion matrix for its predicted results**

In [ ]:
#CODE HERE

In [5]:
import xgboost as xgb

In [6]:
#CODE HERE


my_object_df = df.select_dtypes(include='object')
my_numeric_df = df.select_dtypes(exclude='object')


y = df['Churn']

my_object_df.drop(['customerID', 'Churn'], axis=1, inplace=True)

df_object_dummies = pd.get_dummies(my_object_df,
                                   drop_first=True)

X = pd.concat([my_numeric_df, df_object_dummies], axis=1)


In [7]:
y = np.where(y == 'No', 0, 1)
y

array([0, 0, 1, ..., 0, 1, 0])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=101)

In [9]:
cxgb = xgb.XGBClassifier()

In [10]:
pg = {'booster':['dart', 'gbtree'],
      'learning_rate':[0.005, 0.01, 0.05],
      'n_estimators':[50, 100, 200]}

In [11]:
grid = GridSearchCV(cxgb, pg, cv=10)

In [12]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=Non

In [ ]:
preds = grid.predict(X_test)

In [ ]:
print(grid.get_params)
print(classification_report(y_test, preds))

In [ ]:
plot_confusion_matrix(grid, X_test, y_test)

**TASK: Analyze your results, which model performed best for you?**

In [ ]:
# With base models, we got best performance from an AdaBoostClassifier, but note, we didn't do any gridsearching AND most models performed about the same on the data set.

### Great job!